In this notebook you will find the feature engineering part 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
dim_patient = pd.read_csv('../../../data/dim_patient_donor.csv')

split columns regarding type : for final encoding 

In [8]:
feat = pd.read_csv("../features_config/features_dtypes.csv")
var_continuous = feat[feat.type == 'quantitative'].features.tolist()
var_category = feat[feat.type == 'categorical'].features.tolist()

In [9]:
var_continuous = [ i for i in var_continuous if i in dim_patient.columns]
var_category = [ i for i in var_category if i in dim_patient.columns]

In [10]:
dim_patient = pd.concat([dim_patient, pd.get_dummies(dim_patient[var_category])] , axis = 1)
dim_patient.drop(var_category, axis = 1, inplace = True)


In [13]:
dim_patient.date_transplantation = pd.to_datetime(dim_patient.date_transplantation)
dim_patient['month'] = dim_patient.date_transplantation.dt.month
dim_patient['dayofweek'] = dim_patient.date_transplantation.dt.dayofweek

dim_patient['diff transplantation sortie'] = pd.to_datetime(dim_patient.date_transplantation) - pd.to_datetime(dim_patient.date_sortie_bloc)
dim_patient['diff transplantation sortie']= dim_patient['diff transplantation sortie'].dt.days 

dim_patient.heure_arrivee_rea = dim_patient.heure_arrivee_rea.map(lambda x : x.split(':')[0])
dim_patient.drop(["date_transplantation", "heure_arrivee_rea", "date_sortie_bloc"], axis = 1, inplace = True)


dim_patient = dim_patient.replace("NF", np.nan).fillna(0)
dim_patient[var_continuous] = dim_patient[var_continuous].astype(float)
X = dim_patient.drop("target", axis = 1)
y = pd.get_dummies(dim_patient.target)["successful IE"]

X.drop([X.columns[21], X.columns[22], X.columns[20]], axis = 1, inplace = True)
X.drop("id_patient", axis = 1 , inplace = True)

In [14]:
X.to_csv('../../../data/X.csv', index = False)
pd.DataFrame(data = {'target' : y}).to_csv('../../../data/y.csv', index = False)